<a href="https://colab.research.google.com/github/heros-lab/colaboratory/blob/master/verification_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from google.colab import drive
drive.mount('/content/drive')

work_path = "/content/drive/My Drive/Colab Notebooks"


In [ ]:
class DatasetClass:
    def __init__(self, data_path):
        self.path = data_path
    
    def __reflect_index(self, data, index):
        if index != None:
            data = data[:, index]
        return data
        
    def __load_df(self, data_label):
        data_x = pd.read_csv(f"{self.path}/{data_label}_nx.csv", index_col=0)
        data_y = pd.read_csv(f"{self.path}/{data_label}_ny.csv", index_col=0)
        return data_x, data_y
    
    def __load_data(self, data_label, x_index, y_index):
        data_x, data_y = self.__load_df(data_label)
        data_x = self.__reflect_index(data_x.values, x_index)
        data_y = self.__reflect_index(data_y.values, y_index)
        return data_x, data_y
    
    def __load_stack(self, dataset_list, x_index, y_index):
        for label in dataset_list:
            tmp_x, tmp_y = self.__load_data(label, x_index, y_index)
            if dataset_list.index(label) == 0:
                data_x = tmp_x
                data_y = tmp_y
            else:
                data_x = np.vstack((data_x, tmp_x))
                data_y = np.vstack((data_y, tmp_y))
        return data_x, data_y
    
    def __load_dict(self, dataset_list, x_index, y_index):
        data_x, data_y = {}, {}
        for label in dataset_list:
            tmp_x, tmp_y = self.__load_data(label, x_index, y_index)
            data_x[label] = tmp_x
            data_y[label] = tmp_y
        return data_x, data_y
    
    def get_data(self, dataset_label, x_index=None, y_index=None, dict_type:bool=False):
        if not dict_type:
            if type(dataset_label) == str:
                data_x, data_y = self.__load_data(dataset_label, x_index, y_index)
            else:
                data_x, data_y = self.__load_stack(dataset_label, x_index, y_index)
        else:
            data_x, data_y = self.__load_dict(dataset_label, x_index, y_index)
        return data_x, data_y
    
    def get_dataframe(self, dataset_list):
        data_x = {}
        data_y = {}
        for label in dataset_list:
            tmp_x, tmp_y = self.__load_df(label)
            data_x[label] = tmp_x
            data_y[label] = tmp_y
        return data_x, data_y


In [ ]:
def get_index(model_type):
    if "Conv." in model_type:
        x_index = [0,1,2,3,4,5,6]
    elif "1" in model_type:
        x_index = [0,3,4,6]
    elif "2" in model_type:
        x_index = [1,2,5,6]
    elif "3" in model_type:
        x_index = [1,2,5,6]
    elif "4" in model_type:
        x_index = [0,3,4,5,6]
    else:
        print(f"Error::get_index: input model_type is not exist -> {model_type}")
        x_index = None

    if x_index is None:
        y_index = None
    else:
        y_index = [int(model_type[-1])-1]

    return x_index, y_index


In [ ]:
def create_model(x_dim, y_dim, units):
    act_function = "tanh"
    init_method  = "glorot_uniform"
    model = Sequential()
    model.add(Dense(
            input_dim=x_dim, units=units[0],
            activation=act_function, kernel_initializer=init_method))
    for i in range(len(units) - 1):
        model.add(Dense(
            input_dim=units[i], units=units[i+1],
            activation=act_function, kernel_initializer=init_method))
    model.add(Dense(input_dim=units[-1], units=y_dim))
    model.compile(loss="mse", optimizer=Adam())
    return model


In [ ]:
dataset = DatasetClass(f"{work_path}/data/norms2")

In [ ]:
model_type = "Prop.1"
learn_name = "ms1a"
test_name  = "ms2a"

x_index, y_index = get_index(model_type)
learn_x, learn_y = dataset.get_data(learn_name, x_index, y_index)
test_x, test_y = dataset.get_data(test_name, x_index, y_index, dict_type=True)

print(f"x_index: {x_index}\ny_index: {y_index}")

In [ ]:
num_batch = 512
num_units = [22, 2]

epochs = 200
num_sample = 101

df_loss = pd.DataFrame({})
df_msrs = pd.DataFrame({})
df_free = pd.DataFrame({})
df_step = pd.DataFrame({})

for i in range(num_sample):
    clear_session()
    print(f"\rsampling: #{i:3} / {num_sample-1}", end="")
    model = create_model(learn_x.shape[1], 1, num_units)
    loss = model.fit(learn_x, learn_y, batch_size=num_batch, epochs=epochs, verbose=0)

    df_loss[f"#{i}"] = [np.mean(loss.history["loss"][-5:])]
    df_msrs[f"#{i}"] = model.predict(test_x[test_name[0]], batch_size=num_batch)[:, 0]


In [ ]:
df_loss.to_csv(f"{work_path}/verfs/{model_type}_loss.csv")
df_msrs.to_csv(f"{work_path}/verfs/{model_type}_msrs.csv")
